In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import os
from sklearn.metrics import mutual_info_score

import warnings
warnings.filterwarnings("ignore")

# import xgboost as xgb
import lightgbm as lgb

def load_dataset():
    [t1, t2, t3, t4] = [pd.read_csv(f'../data/train_{i}.csv', parse_dates=["时间"]) for i in range(1, 5)]
    [p1, p2, p3, p4] = [pd.read_csv(f'../data/test_{i}.csv', parse_dates=["时间"]) for i in range(1, 5)]
    [t1, t2, t3, t4] = [t.drop_duplicates().reset_index(drop=True) for t in [t1, t2, t3, t4]]
    return [t1, t2, t3, t4], [p1, p2, p3, p4]

[t1, t2, t3, t4], [p1, p2, p3, p4] = load_dataset()

In [2]:
def get_x(t):
    t['month_cat'] = t["时间"].dt.month
    t['hour_cat'] = t["时间"].dt.month
    t['irr'] = t["辐照度"]
    t['ws'] = t["风速"]
    t['wd_cat'] = pd.cut(t1["风向"], bins=list(range(0, 360, 30)))
    t['pr'] = t["压强"]
    t['hm'] = t["湿度"]
    return t.loc[:, ['month_cat', 'hour_cat', 'irr', 'ws', 'wd_cat', 'pr', 'hm']]

x = get_x(t1)
y = t1["实际功率"].values

In [3]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, GroupKFold, KFold

np.random.seed(42)
kf = KFold(n_splits=4).split(X=x, y=y)

param_grid = {
    'num_leaves': [31, 127],
    'feature_fraction': [0.5, 1.0],
    'bagging_fraction': [0.75, 0.95], 
    'reg_alpha': [0.1, 0.5],
    'num_boost_round': [50, 200, 400],
    'learning_rate': [0.01, 0.005]
}

lgb_estimator = lgb.LGBMRegressor(boosting_type='gbdt',
                                  objective='regression',
                                  bagging_freq=5,
                                  categorical_feature=[0, 1, 4])#,
#                                   early_stopping_rounds=5) # REMOVING THIS ARGUMENT MAKES THE CODE RUN OKAY

gsearch = GridSearchCV(estimator=lgb_estimator, 
                       param_grid=param_grid, 
                       cv=kf) 

lgb_model = gsearch.fit(X=x, y=y)

print(lgb_model.best_params_, lgb_model.best_score_)

{'bagging_fraction': 0.95, 'feature_fraction': 1.0, 'learning_rate': 0.01, 'num_boost_round': 400, 'num_leaves': 31, 'reg_alpha': 0.1} 0.86709308607


In [7]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, GroupKFold, KFold

np.random.seed(42)
kf = KFold(n_splits=4).split(X=x, y=y)

param_grid = {
    'num_leaves': [31, 60],
    'feature_fraction': [0.9, 1.0],
    'bagging_fraction': [0.95], 
    'reg_alpha': [0.1],
    'num_boost_round': [350, 400, 500],
    'learning_rate': [0.01, 0.005]
}

lgb_estimator = lgb.LGBMRegressor(boosting_type='gbdt',
                                  objective='regression',
                                  bagging_freq=5,
                                  categorical_feature=[0, 1, 4])#,
#                                   early_stopping_rounds=5) # REMOVING THIS ARGUMENT MAKES THE CODE RUN OKAY

gsearch = GridSearchCV(estimator=lgb_estimator, 
                       param_grid=param_grid, 
                       cv=kf) 

lgb_model = gsearch.fit(X=x, y=y)

print(lgb_model.best_params_, lgb_model.best_score_)

{'bagging_fraction': 0.95, 'feature_fraction': 1.0, 'learning_rate': 0.01, 'num_boost_round': 400, 'num_leaves': 31, 'reg_alpha': 0.1} 0.86709308607


In [14]:
param = {
    'bagging_fraction': 0.95, 
    'feature_fraction': 1.0, 
    'learning_rate': 0.01, 
    'num_boost_round': 400, 
    'num_leaves': 31, 
    'reg_alpha': 0.1
}

model = lgb.LGBMRegressor(boosting_type='gbdt' ,bagging_freq=5, categorical_feature=[0,1,4], **param)
model.fit(x, y)

LGBMRegressor(bagging_fraction=0.95, bagging_freq=5, boosting_type='gbdt',
       categorical_feature=[0, 1, 4], class_weight=None,
       colsample_bytree=1.0, feature_fraction=1.0, importance_type='split',
       learning_rate=0.01, max_depth=-1, min_child_samples=20,
       min_child_weight=0.001, min_split_gain=0.0, n_estimators=100,
       n_jobs=-1, num_boost_round=400, num_leaves=31, objective=None,
       random_state=None, reg_alpha=0.1, reg_lambda=0.0, silent=True,
       subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [17]:
model.score(x, y)

0.89892748482181362

In [22]:
pred  = model.predict(get_x(p1))

In [26]:
def predict(x, y, pred_x, idx):
    np.random.seed(42)
    kf = KFold(n_splits=4).split(X=x, y=y)

    param_grid = {
        'num_leaves': [31, 60],
        'feature_fraction': [0.9, 1.0],
        'bagging_fraction': [0.95], 
        'reg_alpha': [0.1],
        'num_boost_round': [350, 400, 500],
        'learning_rate': [0.01, 0.005]
    }

    lgb_estimator = lgb.LGBMRegressor(boosting_type='gbdt',
                                      objective='regression',
                                      bagging_freq=5,
                                      categorical_feature=[0, 1, 4])#,
    #                                   early_stopping_rounds=5) # REMOVING THIS ARGUMENT MAKES THE CODE RUN OKAY

    gsearch = GridSearchCV(estimator=lgb_estimator, 
                           param_grid=param_grid, 
                           cv=kf) 

    lgb_model = gsearch.fit(X=x, y=y)

    print(lgb_model.best_params_, lgb_model.best_score_)
    model = lgb.LGBMRegressor(boosting_type='gbdt' ,bagging_freq=5, categorical_feature=[0,1,4], **lgb_model.best_params_)
    model.fit(x, y)
    return pd.DataFrame({'id': idx, 'predicition': model.predict(pred_x)})

In [27]:
res = []
t = [t1, t2, t3, t4]
p = [p1, p2, p3, p4]
for i in range(4):
    x = get_x(t[i])
    y = t[i]["实际功率"]
    pred_x = get_x(p[i])
    idx = p[i]['id']
    res.append(predict(x, y, pred_x, idx))

{'bagging_fraction': 0.95, 'feature_fraction': 1.0, 'learning_rate': 0.01, 'num_boost_round': 400, 'num_leaves': 31, 'reg_alpha': 0.1} 0.86709308607
{'bagging_fraction': 0.95, 'feature_fraction': 1.0, 'learning_rate': 0.01, 'num_boost_round': 350, 'num_leaves': 31, 'reg_alpha': 0.1} 0.648159547875
{'bagging_fraction': 0.95, 'feature_fraction': 1.0, 'learning_rate': 0.01, 'num_boost_round': 350, 'num_leaves': 31, 'reg_alpha': 0.1} 0.789878741422
{'bagging_fraction': 0.95, 'feature_fraction': 0.9, 'learning_rate': 0.01, 'num_boost_round': 500, 'num_leaves': 31, 'reg_alpha': 0.1} 0.808947884092


In [29]:
res = pd.concat(res)

In [33]:
res.to_csv("/home/zhouzr/桌面/submit_20181105.csv", index=None)